## Combinar todos los datasets generados por usuario
En este codigo, me encargo de realizar la combinacion de los datos en grupos de 5 minutos, eso incluye los datos de las comidas, etc. Esto con el fin de obtener las 69 caracteristicas.

In [1074]:
import pandas as pd
import numpy as np

PACIENTE = '016'
PATH_FOLDER = 'F:\\Jesus\\UNSA\\Maestria\\Tesis\\Codigo Test\\PreTesisMasterRepo\Dataset\\'+PACIENTE+'\\'
PATH_FOLDER_DEMOGRAPHIC = 'G:\\Dataset\\big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2\\'

In [1075]:
# Cargar todos los datos
# Leyendo los CSV
acc_values = pd.read_csv(PATH_FOLDER + 'ACC_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
actfis_values = pd.read_csv(PATH_FOLDER + 'actividadFisica_'+PACIENTE+'.csv', engine='python', na_values="not available")
bvp_values = pd.read_csv(PATH_FOLDER + 'BVP_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
dexcom_values = pd.read_csv(PATH_FOLDER + 'Dexcom_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
eda_values = pd.read_csv(PATH_FOLDER + 'EDA_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
hr_values = pd.read_csv(PATH_FOLDER + 'HR_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
ibi_values = pd.read_csv(PATH_FOLDER + 'IBI_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
ibi_vfc_values = pd.read_csv(PATH_FOLDER + 'IBI_5min_hr_data_'+PACIENTE+'.csv', engine='python', na_values="not available")
temp_values = pd.read_csv(PATH_FOLDER + 'TEMP_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
ritmocirc_values = pd.read_csv(PATH_FOLDER + 'ritmo_circadiano_min_'+PACIENTE+'.csv', engine='python', na_values="not available")
food_2h_values = pd.read_csv(PATH_FOLDER + 'Food_2h_'+PACIENTE+'.csv', engine='python', na_values="not available")
food_8h_values = pd.read_csv(PATH_FOLDER + 'Food_8h_'+PACIENTE+'.csv', engine='python', na_values="not available")
food_24h_values = pd.read_csv(PATH_FOLDER + 'Food_24h_'+PACIENTE+'.csv', engine='python', na_values="not available")

In [1076]:
# Leemos la data demografica tambien
demographic_values = pd.read_csv(PATH_FOLDER_DEMOGRAPHIC + 'Demographics.csv', engine='python', na_values="not available")

In [1077]:
dexcom_values.head()

,datetime,glucose_x,glucose_y,nivel
0,2020-07-16 10:45:00,134.0,111.767296,PersNorm
1,2020-07-16 10:50:00,130.0,111.767296,PersNorm
2,2020-07-16 10:55:00,127.0,111.767296,PersNorm
3,2020-07-16 11:00:00,122.0,111.767296,PersNorm
4,2020-07-16 11:05:00,121.0,111.767296,PersNorm


In [1078]:
food_2h_values.head()

,time_begin,calorie_sum,calorie_mean,total_carb_sum,total_carb_mean,sugar_sum,sugar_mean,protein_sum,protein_mean,cantidad_comidas_,media_comidas_
0,2020-02-21 06:00:00,820,410.0,60.6,30.3,6.4,3.2,34.4,17.200000,2,0.523256
1,2020-02-21 08:00:00,280,280.0,56.5,56.5,24.0,24.0,8.0,8.000000,1,0.523256
2,2020-02-21 10:00:00,0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.523256
3,2020-02-21 12:00:00,1077,359.0,81.9,27.3,32.7,10.9,37.7,12.566667,3,0.523256
4,2020-02-21 14:00:00,0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.523256


Como vemos, food tiene unos campos diferentes a los de los tiempo de los otros datasets

In [1079]:
demographic_values.head()

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7


La data demografica contiene el genero y HbA1c el cual es una medida tomada en un laboratorio que tambien influye en la deteccion de la glucosa

* En este caso solo consideraremos tomar la fila de datos correspondiente al ID del paciente

In [1080]:
# Identificador único
id_to_find = int(PACIENTE)  # Convertir a entero

# Filtrar la fila correspondiente al ID
demographic_result = demographic_values.loc[demographic_values['ID'] == id_to_find]

# Mostrar el resultado
print(demographic_result)

    ID Gender  HbA1c
15  16   MALE    5.5


In [1081]:
demographic_result.head()

,ID,Gender,HbA1c
15,16,MALE,5.5


In [1082]:
# Formateamos la data de 2, 8 y 24 horas a 5 minutos
food_2h_values['datetime'] = pd.to_datetime(food_2h_values['time_begin'])
food_8h_values['datetime'] = pd.to_datetime(food_8h_values['time_begin'])
food_24h_values['datetime'] = pd.to_datetime(food_24h_values['time_begin'])

food_2h_values = food_2h_values.drop(columns=['time_begin'])
food_8h_values = food_8h_values.drop(columns=['time_begin','calorie_sum','calorie_mean','total_carb_sum','total_carb_mean','sugar_sum','sugar_mean','protein_sum','protein_mean'])
food_24h_values = food_24h_values.drop(columns=['time_begin','calorie_sum','calorie_mean','total_carb_sum','total_carb_mean','sugar_sum','sugar_mean','protein_sum','protein_mean'])

food_2h_values = food_2h_values.fillna(0)
food_8h_values = food_8h_values.fillna(0)
food_24h_values = food_24h_values.fillna(0)

### Aqui tiene que liderar dexcom_values debido a que contiene el flag de glucosa donde indicaremos si es alto, medio o bajo en glucosa

In [1083]:
dexcom_values.head()

,datetime,glucose_x,glucose_y,nivel
0,2020-07-16 10:45:00,134.0,111.767296,PersNorm
1,2020-07-16 10:50:00,130.0,111.767296,PersNorm
2,2020-07-16 10:55:00,127.0,111.767296,PersNorm
3,2020-07-16 11:00:00,122.0,111.767296,PersNorm
4,2020-07-16 11:05:00,121.0,111.767296,PersNorm


**Añadir los valores demograficos**

In [1084]:
# Realizar la combinación de los DataFrames usando pd.merge()
# Comenzamos con el primer DataFrame
dexcom_values = dexcom_values.rename(columns={'glucose_x':'glucose','glucose_y':'glucose_24h_mean'})
final_df = dexcom_values
demographic_values_row = demographic_result.iloc[0]

final_df['gender'] = demographic_values_row['Gender']
final_df['hba1c'] = demographic_values_row['HbA1c']
final_df['id'] = demographic_values_row['ID']
# Obtener los valores de la fila 1

final_df.head()

,datetime,glucose,glucose_24h_mean,nivel,gender,hba1c,id
0,2020-07-16 10:45:00,134.0,111.767296,PersNorm,MALE,5.5,16
1,2020-07-16 10:50:00,130.0,111.767296,PersNorm,MALE,5.5,16
2,2020-07-16 10:55:00,127.0,111.767296,PersNorm,MALE,5.5,16
3,2020-07-16 11:00:00,122.0,111.767296,PersNorm,MALE,5.5,16
4,2020-07-16 11:05:00,121.0,111.767296,PersNorm,MALE,5.5,16


### Empezamos por validar que datos necesitamos para cada dataset
Para este punto vamos a evaluar cada uno de los datasets y verificar que columnas necesitamos de cada uno

**EDA**

In [1085]:
eda_values.head()

,datetime,mean,median,max,min,std,total_peaks,mean_peaks,estres,q1,q3
0,2020-07-16 09:25:00,0.448949,0.064036,3.570595,0.000000,0.803402,8,0.035088,1,0.005123,0.335883
1,2020-07-16 09:30:00,0.326691,0.372047,0.653164,0.016649,0.165484,1,0.000833,1,0.139598,0.452092
2,2020-07-16 09:35:00,0.631900,0.631659,0.764853,0.506149,0.051198,0,0.000000,0,0.591957,0.670080
3,2020-07-16 09:40:00,1.030635,1.036434,1.396314,0.658554,0.194995,3,0.002500,1,0.858042,1.190439
4,2020-07-16 09:45:00,1.049776,1.044758,1.259278,0.846888,0.083042,1,0.000833,1,0.986486,1.117119


In [1086]:
# Eliminemos las columnas que en este caso no necesitamos
eda_values = eda_values.drop(columns=['total_peaks','mean_peaks'])
# Renombrar las columnas para mayor claridad
# Agregando 'eda_' como prefijo
eda_values.rename(columns=lambda x: f'eda_{x}' if x != 'datetime' else x, inplace=True)
eda_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1866 entries, 0 to 1865
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    1866 non-null   object 
 1   eda_mean    1866 non-null   float64
 2   eda_median  1866 non-null   float64
 3   eda_max     1866 non-null   float64
 4   eda_min     1866 non-null   float64
 5   eda_std     1866 non-null   float64
 6   eda_estres  1866 non-null   int64  
 7   eda_q1      1866 non-null   float64
 8   eda_q3      1866 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 131.3+ KB


**ACC**

In [1087]:
acc_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-07-16 09:25:00,65.035743,63.269266,167.014969,14.456832,11.721119,60.514461,67.448126
1,2020-07-16 09:30:00,62.730299,62.801274,133.768457,19.339080,3.857899,62.120850,63.269266
2,2020-07-16 09:35:00,62.568438,62.513998,139.907112,25.019992,3.254793,61.862751,63.063460
3,2020-07-16 09:40:00,62.401744,62.225397,130.326513,31.701735,3.242671,61.717096,62.841069
4,2020-07-16 09:45:00,62.548595,62.465991,109.402925,28.160256,2.355077,61.983869,62.880840


In [1088]:
# Renombrar las columnas para mayor claridad
# Agregando 'acc_' como prefijo
acc_values.rename(columns=lambda x: f'acc_{x}' if x != 'datetime' else x, inplace=True)
acc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2173 non-null   object 
 1   acc_mean    1866 non-null   float64
 2   acc_median  1866 non-null   float64
 3   acc_max     1866 non-null   float64
 4   acc_min     1866 non-null   float64
 5   acc_std     1866 non-null   float64
 6   acc_q1      1866 non-null   float64
 7   acc_q3      1866 non-null   float64
dtypes: float64(7), object(1)
memory usage: 135.9+ KB


**Actividad Fisica**

In [1089]:
actfis_values.head()

,datetime,mean_x,median_x,max_x,min_x,std_x,q1_x,q3_x,mean_y,median_y,max_y,min_y,std_y,q1_y,q3_y,per_act
0,2020-07-16 09:25:00,65.035743,63.269266,167.014969,14.456832,11.721119,60.514461,67.448126,92.710426,94.300,98.04,71.00,5.689590,90.2050,97.2550,0
1,2020-07-16 09:30:00,62.730299,62.801274,133.768457,19.339080,3.857899,62.120850,63.269266,78.698367,78.385,89.25,64.33,5.966452,74.7200,84.7700,0
2,2020-07-16 09:35:00,62.568438,62.513998,139.907112,25.019992,3.254793,61.862751,63.063460,72.131633,73.325,80.73,62.18,5.619535,66.8775,77.0825,0
3,2020-07-16 09:40:00,62.401744,62.225397,130.326513,31.701735,3.242671,61.717096,62.841069,80.821033,77.780,96.55,73.35,6.748584,76.2525,84.9550,0
4,2020-07-16 09:45:00,62.548595,62.465991,109.402925,28.160256,2.355077,61.983869,62.880840,75.150100,75.125,81.53,68.90,2.996422,73.8175,77.0200,0


In [1090]:
actfis_values_final = actfis_values[['datetime', 'per_act']]
actfis_values_final.rename(columns=lambda x: f'actfis{x}' if x != 'datetime' else x, inplace=True)
actfis_values_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1866 entries, 0 to 1865
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   datetime       1866 non-null   object
 1   actfisper_act  1866 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.3+ KB


C:\Users\Alpha\AppData\Local\Temp\ipykernel_22624\687457855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actfis_values_final.rename(columns=lambda x: f'actfis{x}' if x != 'datetime' else x, inplace=True)


**BVP**

In [1091]:
bvp_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-07-16 09:25:00,0.075661,-0.515,1087.16,-800.87,202.016886,-86.1200,81.6100
1,2020-07-16 09:30:00,0.121937,2.545,887.80,-869.75,100.128266,-29.6425,29.5550
2,2020-07-16 09:35:00,-0.157578,2.220,366.76,-465.28,56.261485,-20.6025,22.3550
3,2020-07-16 09:40:00,0.036537,0.890,1108.86,-1143.69,76.742150,-21.2150,20.9225
4,2020-07-16 09:45:00,0.073001,3.400,584.70,-768.77,55.487780,-22.7425,23.3700


In [1092]:
# Renombrar las columnas para mayor claridad
# Agregando 'bvp_' como prefijo
bvp_values.rename(columns=lambda x: f'bvp_{x}' if x != 'datetime' else x, inplace=True)
bvp_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2173 non-null   object 
 1   bvp_mean    1866 non-null   float64
 2   bvp_median  1866 non-null   float64
 3   bvp_max     1866 non-null   float64
 4   bvp_min     1866 non-null   float64
 5   bvp_std     1866 non-null   float64
 6   bvp_q1      1866 non-null   float64
 7   bvp_q3      1866 non-null   float64
dtypes: float64(7), object(1)
memory usage: 135.9+ KB


**HR**

In [1093]:
hr_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-07-16 09:25:00,92.710426,94.300,98.04,71.00,5.689590,90.2050,97.2550
1,2020-07-16 09:30:00,78.698367,78.385,89.25,64.33,5.966452,74.7200,84.7700
2,2020-07-16 09:35:00,72.131633,73.325,80.73,62.18,5.619535,66.8775,77.0825
3,2020-07-16 09:40:00,80.821033,77.780,96.55,73.35,6.748584,76.2525,84.9550
4,2020-07-16 09:45:00,75.150100,75.125,81.53,68.90,2.996422,73.8175,77.0200


In [1094]:
# Renombrar las columnas para mayor claridad
# Agregando 'hr_' como prefijo
hr_values.rename(columns=lambda x: f'hr_{x}' if x != 'datetime' else x, inplace=True)
hr_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   2173 non-null   object 
 1   hr_mean    1866 non-null   float64
 2   hr_median  1866 non-null   float64
 3   hr_max     1866 non-null   float64
 4   hr_min     1866 non-null   float64
 5   hr_std     1866 non-null   float64
 6   hr_q1      1866 non-null   float64
 7   hr_q3      1866 non-null   float64
dtypes: float64(7), object(1)
memory usage: 135.9+ KB


**IBI**

In [1095]:
ibi_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-07-16 09:30:00,0.776749,0.781286,0.906291,0.703157,0.032791,0.750034,0.796911
1,2020-07-16 09:35:00,0.785514,0.781286,0.953169,0.656280,0.050713,0.750034,0.812537
2,2020-07-16 09:40:00,0.773977,0.781286,1.000046,0.578151,0.073439,0.734409,0.796911
3,2020-07-16 09:45:00,0.798779,0.796911,0.968794,0.640654,0.048668,0.765660,0.828163
4,2020-07-16 09:50:00,0.793604,0.796911,0.984420,0.531274,0.066225,0.765660,0.828163


In [1096]:
# Renombrar las columnas para mayor claridad
# Agregando 'ibi_' como prefijo
ibi_values.rename(columns=lambda x: f'ibi_{x}' if x != 'datetime' else x, inplace=True)
ibi_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2172 entries, 0 to 2171
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2172 non-null   object 
 1   ibi_mean    2172 non-null   float64
 2   ibi_median  2172 non-null   float64
 3   ibi_max     2172 non-null   float64
 4   ibi_min     2172 non-null   float64
 5   ibi_std     2172 non-null   float64
 6   ibi_q1      1824 non-null   float64
 7   ibi_q3      1824 non-null   float64
dtypes: float64(7), object(1)
memory usage: 135.9+ KB


**TEMP**

In [1097]:
temp_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-07-16 09:25:00,28.031053,28.37,30.33,23.71,2.101171,26.71,30.170
1,2020-07-16 09:30:00,31.415867,31.44,31.87,30.33,0.363143,31.19,31.710
2,2020-07-16 09:35:00,31.871467,31.87,32.13,31.61,0.151529,31.73,32.000
3,2020-07-16 09:40:00,32.197567,31.99,33.34,31.61,0.532143,31.77,32.665
4,2020-07-16 09:45:00,33.419567,33.45,33.63,33.09,0.141820,33.34,33.530


In [1098]:
# Renombrar las columnas para mayor claridad
# Agregando 'temp_' como prefijo
temp_values.rename(columns=lambda x: f'temp_{x}' if x != 'datetime' else x, inplace=True)
temp_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   datetime     2173 non-null   object 
 1   temp_mean    1866 non-null   float64
 2   temp_median  1866 non-null   float64
 3   temp_max     1866 non-null   float64
 4   temp_min     1866 non-null   float64
 5   temp_std     1866 non-null   float64
 6   temp_q1      1866 non-null   float64
 7   temp_q3      1866 non-null   float64
dtypes: float64(7), object(1)
memory usage: 135.9+ KB


**VFC**

In [1099]:
ibi_vfc_values.head()

,datetime,ibi,MeanRR,MeanHR,MinHR,MaxHR,SDNN,RMSSD,NNx,pNNx,PowerVLF,PowerLF,PowerHF,PowerTotal,LF/HF,PeakVLF,PeakLF,PeakHF,FractionLF,FractionHF
0,2020-07-16 09:30:00,0.776749,778.1,77.1,73.6,78.8,11.7,49.8,24.0,26.1,39.84,57.64,334.60,432.08,0.17,0.02,0.14,0.34,14.70,85.30
1,2020-07-16 09:35:00,0.785514,787.8,76.2,73.1,80.3,15.3,73.7,27.0,32.1,95.94,288.62,673.65,1058.20,0.43,0.03,0.09,0.39,29.99,70.01
2,2020-07-16 09:40:00,0.773977,779.3,77.1,69.6,81.7,33.8,86.7,25.0,41.0,240.52,762.92,653.33,1656.78,1.17,0.02,0.09,0.30,53.87,46.13
3,2020-07-16 09:45:00,0.798779,797.9,75.2,71.6,79.5,18.3,66.9,59.0,37.3,101.57,249.34,788.83,1139.74,0.32,0.03,0.05,0.36,24.02,75.98
4,2020-07-16 09:50:00,0.793604,801.0,75.0,71.9,86.7,24.2,83.4,50.0,36.8,131.81,318.40,1115.08,1565.28,0.29,0.02,0.11,0.34,22.21,77.79


In [1100]:
# Renombrar las columnas para mayor claridad
# Agregando 'vfc_' como prefijo
ibi_vfc_values.rename(columns=lambda x: f'vfc_{x}' if x != 'datetime' else x, inplace=True)
ibi_vfc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime        1824 non-null   object 
 1   vfc_ ibi        1824 non-null   float64
 2   vfc_MeanRR      1824 non-null   float64
 3   vfc_MeanHR      1824 non-null   float64
 4   vfc_MinHR       1824 non-null   float64
 5   vfc_MaxHR       1824 non-null   float64
 6   vfc_SDNN        1824 non-null   float64
 7   vfc_RMSSD       1824 non-null   float64
 8   vfc_NNx         1824 non-null   float64
 9   vfc_pNNx        1824 non-null   float64
 10  vfc_PowerVLF    1824 non-null   float64
 11  vfc_PowerLF     1824 non-null   float64
 12  vfc_PowerHF     1824 non-null   float64
 13  vfc_PowerTotal  1824 non-null   float64
 14  vfc_LF/HF       1813 non-null   float64
 15  vfc_PeakVLF     1824 non-null   float64
 16  vfc_PeakLF      1824 non-null   float64
 17  vfc_PeakHF      1824 non-null   f

In [1101]:
columns_to_drop_vfc = ['vfc_ ibi','vfc_MeanRR','vfc_PowerVLF','vfc_PowerLF','vfc_PowerHF','vfc_PowerTotal','vfc_LF/HF','vfc_PeakVLF','vfc_PeakLF','vfc_PeakHF','vfc_FractionLF','vfc_FractionHF']
ibi_vfc_values = ibi_vfc_values.drop(columns=columns_to_drop_vfc)
ibi_vfc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    1824 non-null   object 
 1   vfc_MeanHR  1824 non-null   float64
 2   vfc_MinHR   1824 non-null   float64
 3   vfc_MaxHR   1824 non-null   float64
 4   vfc_SDNN    1824 non-null   float64
 5   vfc_RMSSD   1824 non-null   float64
 6   vfc_NNx     1824 non-null   float64
 7   vfc_pNNx    1824 non-null   float64
dtypes: float64(7), object(1)
memory usage: 114.1+ KB


**Ritmo Circadiano**

In [1102]:
ritmocirc_values.head()

,datetime,magnitude_mean,magnitude_std,hr_mean,hr_std,indicator_ritmo_circadiano,date,hr_mean_indicator,hr_std_indicator,magnitude_mean_indicator,magnitude_std_indicator,sum_indicator
0,2020-07-16 09:25:00,65.035743,11.721119,92.710426,5.689590,1,2020-07-16,0,1,0,0,1
1,2020-07-16 09:30:00,62.730299,3.857899,78.698367,5.966452,0,2020-07-16,1,1,1,1,4
2,2020-07-16 09:35:00,62.568438,3.254793,72.131633,5.619535,0,2020-07-16,1,1,1,1,4
3,2020-07-16 09:40:00,62.401744,3.242671,80.821033,6.748584,0,2020-07-16,1,1,1,1,4
4,2020-07-16 09:45:00,62.548595,2.355077,75.150100,2.996422,0,2020-07-16,1,1,1,1,4


In [1103]:
# Renombrar las columnas para mayor claridad
# Agregando 'ritcir_' como prefijo
ritmocirc_values.rename(columns=lambda x: f'ritcir_{x}' if x != 'datetime' else x, inplace=True)
ritmocirc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   datetime                           2173 non-null   object 
 1   ritcir_magnitude_mean              2173 non-null   float64
 2   ritcir_magnitude_std               2173 non-null   float64
 3   ritcir_hr_mean                     2173 non-null   float64
 4   ritcir_hr_std                      2173 non-null   float64
 5   ritcir_indicator_ritmo_circadiano  2173 non-null   int64  
 6   ritcir_date                        2173 non-null   object 
 7   ritcir_hr_mean_indicator           2173 non-null   int64  
 8   ritcir_hr_std_indicator            2173 non-null   int64  
 9   ritcir_magnitude_mean_indicator    2173 non-null   int64  
 10  ritcir_magnitude_std_indicator     2173 non-null   int64  
 11  ritcir_sum_indicator               2173 non-null   int64

In [1104]:
columns_to_drop_ritcir = ['ritcir_magnitude_mean','ritcir_magnitude_std','ritcir_hr_mean','ritcir_hr_std','ritcir_date','ritcir_sum_indicator','ritcir_magnitude_mean_indicator','ritcir_hr_mean_indicator','ritcir_hr_std_indicator','ritcir_magnitude_std_indicator']
ritmocirc_values = ritmocirc_values.drop(columns=columns_to_drop_ritcir)
ritmocirc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 2 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   datetime                           2173 non-null   object
 1   ritcir_indicator_ritmo_circadiano  2173 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.1+ KB


**FOOD**

In [1105]:
food_2h_values.head()

,calorie_sum,calorie_mean,total_carb_sum,total_carb_mean,sugar_sum,sugar_mean,protein_sum,protein_mean,cantidad_comidas_,media_comidas_,datetime
0,820,410.0,60.6,30.3,6.4,3.2,34.4,17.200000,2,0.523256,2020-02-21 06:00:00
1,280,280.0,56.5,56.5,24.0,24.0,8.0,8.000000,1,0.523256,2020-02-21 08:00:00
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.523256,2020-02-21 10:00:00
3,1077,359.0,81.9,27.3,32.7,10.9,37.7,12.566667,3,0.523256,2020-02-21 12:00:00
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.523256,2020-02-21 14:00:00


In [1106]:
# Renombrar las columnas para mayor claridad
# Agregando 'food2h_' como prefijo
food_2h_values.rename(columns=lambda x: f'food2h_{x}' if x != 'datetime' else x, inplace=True)

# Agregando 'food8h_' como prefijo
food_8h_values.rename(columns=lambda x: f'food8h_{x}' if x != 'datetime' else x, inplace=True)

# Agregando 'food24h_' como prefijo
food_24h_values.rename(columns=lambda x: f'food24h_{x}' if x != 'datetime' else x, inplace=True)
food_24h_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   food24h_cantidad_comidas_  8 non-null      int64         
 1   food24h_media_comidas_     8 non-null      float64       
 2   datetime                   8 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 324.0 bytes


### Combinacion
Para este paso ya tenemos los dataframes listos para ser combinados

In [1107]:
# Revisamos el dataframe mas importante
final_df.head()

,datetime,glucose,glucose_24h_mean,nivel,gender,hba1c,id
0,2020-07-16 10:45:00,134.0,111.767296,PersNorm,MALE,5.5,16
1,2020-07-16 10:50:00,130.0,111.767296,PersNorm,MALE,5.5,16
2,2020-07-16 10:55:00,127.0,111.767296,PersNorm,MALE,5.5,16
3,2020-07-16 11:00:00,122.0,111.767296,PersNorm,MALE,5.5,16
4,2020-07-16 11:05:00,121.0,111.767296,PersNorm,MALE,5.5,16


In [1108]:
# Establecer la columna 'datetime' como índice
acc_values.set_index('datetime', inplace=True)
actfis_values_final.set_index('datetime', inplace=True)
bvp_values.set_index('datetime', inplace=True)
dexcom_values.set_index('datetime', inplace=True)
eda_values.set_index('datetime', inplace=True)
hr_values.set_index('datetime', inplace=True)
ibi_values.set_index('datetime', inplace=True)
ibi_vfc_values.set_index('datetime', inplace=True)
temp_values.set_index('datetime', inplace=True)
ritmocirc_values.set_index('datetime', inplace=True)
food_2h_values.set_index('datetime', inplace=True)
food_8h_values.set_index('datetime', inplace=True)
food_24h_values.set_index('datetime', inplace=True)


In [1109]:
# Resamplear los dataframes de menor frecuencia a 5 minutos
food_2h_resampled = food_2h_values.resample('5T').ffill()
food_8h_resampled = food_8h_values.resample('5T').ffill()
food_24h_resampled = food_24h_values.resample('5T').ffill()
food_24h_resampled.head()

,food24h_cantidad_comidas_,food24h_media_comidas_
datetime,,
2020-02-21 00:00:00,10,5.625
2020-02-21 00:05:00,10,5.625
2020-02-21 00:10:00,10,5.625
2020-02-21 00:15:00,10,5.625
2020-02-21 00:20:00,10,5.625


In [1110]:
# Lista de DataFrames a combinar
dataframes_to_merge = [final_df
,acc_values
,actfis_values_final
,bvp_values
,eda_values
,hr_values
,ibi_values
,ibi_vfc_values
,temp_values
,ritmocirc_values
,food_2h_resampled
,food_8h_resampled
,food_24h_resampled]



In [1111]:
final_df = pd.concat(dataframes_to_merge, axis=1)

In [1112]:
final_df = final_df.fillna(0)
final_df.head()

,glucose,glucose_24h_mean,nivel,gender,hba1c,id,acc_mean,acc_median,acc_max,acc_min,...,food2h_sugar_sum,food2h_sugar_mean,food2h_protein_sum,food2h_protein_mean,food2h_cantidad_comidas_,food2h_media_comidas_,food8h_cantidad_comidas_,food8h_media_comidas_,food24h_cantidad_comidas_,food24h_media_comidas_
datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-16 10:45:00,134.0,111.767296,PersNorm,MALE,5.5,16.0,62.857097,62.249498,134.048499,30.675723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-16 10:50:00,130.0,111.767296,PersNorm,MALE,5.5,16.0,64.245831,63.091204,154.938698,13.038405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-16 10:55:00,127.0,111.767296,PersNorm,MALE,5.5,16.0,63.132467,62.872888,139.122248,8.774964,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-16 11:00:00,122.0,111.767296,PersNorm,MALE,5.5,16.0,62.987030,62.841069,118.304691,16.911535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-16 11:05:00,121.0,111.767296,PersNorm,MALE,5.5,16.0,62.842033,62.425956,146.989796,32.526912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1113]:
final_df = final_df[final_df['glucose'] != 0]
print(final_df.columns)

Index(['glucose', 'glucose_24h_mean', 'nivel', 'gender', 'hba1c', 'id',
       'acc_mean', 'acc_median', 'acc_max', 'acc_min', 'acc_std', 'acc_q1',
       'acc_q3', 'actfisper_act', 'bvp_mean', 'bvp_median', 'bvp_max',
       'bvp_min', 'bvp_std', 'bvp_q1', 'bvp_q3', 'eda_mean', 'eda_median',
       'eda_max', 'eda_min', 'eda_std', 'eda_estres', 'eda_q1', 'eda_q3',
       'hr_mean', 'hr_median', 'hr_max', 'hr_min', 'hr_std', 'hr_q1', 'hr_q3',
       'ibi_mean', 'ibi_median', 'ibi_max', 'ibi_min', 'ibi_std', 'ibi_q1',
       'ibi_q3', 'vfc_MeanHR', 'vfc_MinHR', 'vfc_MaxHR', 'vfc_SDNN',
       'vfc_RMSSD', 'vfc_NNx', 'vfc_pNNx', 'temp_mean', 'temp_median',
       'temp_max', 'temp_min', 'temp_std', 'temp_q1', 'temp_q3',
       'ritcir_indicator_ritmo_circadiano', 'food2h_calorie_sum',
       'food2h_calorie_mean', 'food2h_total_carb_sum',
       'food2h_total_carb_mean', 'food2h_sugar_sum', 'food2h_sugar_mean',
       'food2h_protein_sum', 'food2h_protein_mean', 'food2h_cantidad_comidas_

In [1114]:
final_df.to_csv(PACIENTE+'_dataset.csv')